In [106]:
import pandas as pd
import numpy as np

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime, date

%matplotlib inline
sns.set_style('white')


# 3.6.4 [Housing Prices](https://courses.thinkful.com/data-201v1/project/3.6.4)
Using this [Kaggle data](https://www.kaggle.com/anthonypino/melbourne-housing-market) create a model to predict a house's value. We want to be able to understand what creates value in a house, as though we were a real estate developer.

Type: H=House, U=Unit, T=Townhouse
Method: S - property sold; 
        SP - property sold prior; 
        PI - property passed in; 
        VB - vendor bid; 
        SA - sold after auction; 
Property Count: Number of properties that exist in the suburb.
Rooms: Number of rooms
SellerG: Sellers Name
Date: Date sold
Distance: Distance from CBD in km
Postcode: 
Regionname: Name of regin General Region (West, North West, North, North east ...etc)
Council Area: Governing council for the area




In [77]:
df_housing = pd.DataFrame(pd.read_csv('melbourne_housing.csv'))

# Remove instances where price is null
df_housing = df_housing.dropna(axis=0, subset=['Price'])
df_housing = df_housing[df_housing['Method']=='S']


def convert_datetime(date):
    date_format = '%d/%m/%Y'
    date = datetime.strptime(date, date_format)
    return date
def house_type(house_type):
    if house_type=='h':
        return 'house'
    elif house_type=='u':
        return 'unit'
    elif house_type=='t':
        return 'townhouse'

df_housing['Type'] = df_housing['Type'].apply(lambda x: house_type(x))
df_housing['Date'] = df_housing['Date'].apply(lambda x: convert_datetime(x))

df_housing['sold_dow'] = df_housing['Date'].dt.day_name()
df_housing['sold_month'] = df_housing['Date'].dt.month_name()

In [78]:
df_housing.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea,sold_dow,sold_month
0,Abbotsford,49 Lithgow St,3,house,1490000.0,S,Jellis,2017-04-01,3067,Northern Metropolitan,4019,3.0,Yarra City Council,Saturday,April
1,Abbotsford,59A Turner St,3,house,1220000.0,S,Marshall,2017-04-01,3067,Northern Metropolitan,4019,3.0,Yarra City Council,Saturday,April
2,Abbotsford,119B Yarra St,3,house,1420000.0,S,Nelson,2017-04-01,3067,Northern Metropolitan,4019,3.0,Yarra City Council,Saturday,April
3,Aberfeldie,68 Vida St,3,house,1515000.0,S,Barry,2017-04-01,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council,Saturday,April
4,Airport West,92 Clydesdale Rd,2,house,670000.0,S,Nelson,2017-04-01,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council,Saturday,April


In [80]:
#[2*a for a in x if a % 2 == 1]
unique_values=[]
[unique_values.append(len(df_housing[x].unique())) for x in df_housing.columns]
feature_list=df_housing.columns
categories=pd.DataFrame(data={'features':feature_list, 'unique_values':unique_values})
print(categories)

         features  unique_values
0          Suburb            356
1         Address          28760
2           Rooms             14
3            Type              3
4           Price           3103
5          Method              1
6         SellerG            338
7            Date            112
8        Postcode            214
9      Regionname              8
10  Propertycount            347
11       Distance            173
12    CouncilArea             34
13       sold_dow              5
14     sold_month             12


In [79]:
df_housing.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Postcode', 'Regionname', 'Propertycount', 'Distance',
       'CouncilArea', 'sold_dow', 'sold_month'],
      dtype='object')

In [102]:
feature_columns= ['Rooms', 'Type', 'sold_dow', 'sold_month', 'Regionname', 'Distance']
X = df_housing[feature_columns]
X = pd.get_dummies(X)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

y = df_housing['Price']

### Hypothesis for housing prices:
* Certain regions and suburbs will have a 


In [109]:
from sklearn.model_selection import cross_val_score

In [115]:
from sklearn.ensemble import RandomForestRegressor

In [132]:
rfr = RandomForestRegressor(max_depth=4, n_estimators=10)
rfr.fit(X_scaled,y)

df_features = pd.DataFrame(data={'features':X.columns,
                                 'feature_importance':rfr.feature_importances_}).sort_values(
    by='feature_importance', ascending=False).reset_index(drop=True)

In [134]:
df_features.head(7)

,features,feature_importance
0,Type_house,0.347382
1,Regionname_Southern Metropolitan,0.237963
2,Rooms,0.180616
3,Distance,0.150414
4,Regionname_Eastern Metropolitan,0.074621
5,Regionname_South-Eastern Metropolitan,0.008833
6,Type_townhouse,0.000170


In [129]:
df_features.iloc[0:6,0]

0                               Type_house
1         Regionname_Southern Metropolitan
2                                    Rooms
3                                 Distance
4          Regionname_Eastern Metropolitan
5    Regionname_South-Eastern Metropolitan
Name: features, dtype: object

In [130]:
feature_columns= ['Rooms', 'Type', 'Regionname', 'Distance']
X = df_housing[feature_columns]
X = pd.get_dummies(X)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

y = df_housing['Price']

In [131]:
lr = LinearRegression()
print(cross_val_score(lr,X,y,cv=5))

[0.51242021 0.52444574 0.50948968 0.52505541 0.50339107]
